In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from treeinterpreter import treeinterpreter as ti
import shap
from scipy.special import cbrt

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from openpyxl import load_workbook,Workbook

# Getting data

In [3]:
os.chdir('../pickles')

In [4]:
numerical = pickle.load(open('numerical.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))
conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
imputation_cols = pickle.load(open('imputation_cols.pickle','rb'))
model_columns = pickle.load(open('model_columns','rb'))
lr_model_columns = pickle.load(open('lr_model_columns.pickle','rb'))
order = pickle.load(open('order.pickle','rb'))

In [5]:
os.chdir('../Statistics/')

In [6]:
df_val = pd.read_excel('Q1_predictions.xlsx',sheet_name='data',header=0)

In [7]:
df_val.columns

Index(['Prev_Close(in 10^5)', 'High_1d(in 10^5)', 'Low_1d(in 10^5)',
       'Last_1d(in 10^5)', 'Close', 'VWAP_1d(in 10^5)', 'Volume_1d(in 10^9)',
       'Turnover_1d(in 10^11)', 'Trades_1d(in 10^7)',
       'Deliverable_Volume_1d(in 10^9)',
       ...
       'return_5d_500_sqr', 'Voltality_500_log', 'Voltality_log',
       'RF_Prediction', 'GBM_Prediction', 'LR_Prediction', 'Symbol', 'Date',
       'Open', 'ensemble_prediction'],
      dtype='object', length=219)

In [8]:
os.chdir('../Input_Data/')

# Reason Codes for Random Forest Model

In [9]:
os.chdir('../Models/')

In [10]:
RF = pickle.load(open('RF.pickle','rb'))

In [11]:
df_val.reset_index(inplace=True)

In [12]:
df_val.columns

Index(['index', 'Prev_Close(in 10^5)', 'High_1d(in 10^5)', 'Low_1d(in 10^5)',
       'Last_1d(in 10^5)', 'Close', 'VWAP_1d(in 10^5)', 'Volume_1d(in 10^9)',
       'Turnover_1d(in 10^11)', 'Trades_1d(in 10^7)',
       ...
       'return_5d_500_sqr', 'Voltality_500_log', 'Voltality_log',
       'RF_Prediction', 'GBM_Prediction', 'LR_Prediction', 'Symbol', 'Date',
       'Open', 'ensemble_prediction'],
      dtype='object', length=220)

In [14]:
def reason_code_rf(model,data,target='Close'):
    predictors = lr_model_columns
    df_reasons = pd.DataFrame()
    prediction, bias, contributions = ti.predict(model,np.array(data[lr_model_columns]))
    df_reasons['Symbol'] = data['Symbol']
    df_reasons['Date'] = data['Date']
    df_reasons['Open Price'] = data['Open']
    df_reasons['Prev Close'] = data['Prev_Close(in 10^5)']*(10**5)
    df_reasons['bias'] = bias
    df_reasons['Actual_Value'] = data['Close']
    df_reasons['Predicted_Value']=prediction
    df_reasons['Expected Profit'] = df_reasons['Predicted_Value']-df_reasons['Prev Close']
    df_reasons['Actual Profit'] = df_reasons['Actual_Value'] - df_reasons['Open Price']
    df_reasons['Profit Diff'] = df_reasons['Expected Profit'] - df_reasons['Actual Profit']
    contributions = pd.DataFrame(contributions)
    contributions['mrc'] = contributions.apply(lambda x: ','.join([a+':'+str(b) for (a,b) in sorted(zip(lr_model_columns,x),key = lambda x: x[1])]),axis=1)
    df_reasons['mrc'] = contributions['mrc']
    
    return df_reasons

In [15]:
df_reasons = reason_code_rf(RF,df_val)

In [16]:
df_reasons

,Symbol,Date,Open Price,Prev Close,bias,Actual_Value,Predicted_Value,Expected Profit,Actual Profit,Profit Diff,mrc
0,ADANIPORTS,2019-01-01,386.30,387.70,1370.471809,389.00,381.837048,-5.862952,2.70,-8.562952,"Prev_Close(in 10^5):-988.633971863707,return_1..."
1,ADANIPORTS,2019-01-02,388.00,389.00,1370.471809,381.30,387.765472,-1.234528,-6.70,5.465472,"Prev_Close(in 10^5):-982.7065996342455,return_..."
2,ADANIPORTS,2019-01-03,382.40,381.30,1370.471809,378.10,380.286191,-1.013809,-4.30,3.286191,"Prev_Close(in 10^5):-990.1820234869239,return_..."
3,ADANIPORTS,2019-01-04,378.65,378.10,1370.471809,382.15,379.932977,1.832977,3.50,-1.667023,"Prev_Close(in 10^5):-990.5347442091077,return_..."
4,ADANIPORTS,2019-01-07,385.00,382.15,1370.471809,381.65,380.594916,-1.555084,-3.35,1.794916,"Prev_Close(in 10^5):-989.8826183615503,return_..."
5,ADANIPORTS,2019-01-08,383.00,381.65,1370.471809,378.75,380.496919,-1.153081,-4.25,3.096919,"Prev_Close(in 10^5):-989.977116918424,return_2..."
6,ADANIPORTS,2019-01-09,382.00,378.75,1370.471809,377.30,379.959722,1.209722,-4.70,5.909722,"Prev_Close(in 10^5):-990.5135284981667,return_..."
7,ADANIPORTS,2019-01-10,377.95,377.30,1370.471809,380.25,378.876702,1.576702,2.30,-0.723298,"Prev_Close(in 10^5):-991.5971825043473,return_..."
8,ADANIPORTS,2019-01-11,380.30,380.25,1370.471809,377.15,380.009397,-0.240603,-3.15,2.909397,"Prev_Close(in 10^5):-990.4660514175573,return_..."
9,ADANIPORTS,2019-01-14,377.00,377.15,1370.471809,374.45,378.561846,1.411846,-2.55,3.961846,"Prev_Close(in 10^5):-991.9132927667362,return_..."


In [17]:
os.chdir('../Statistics/')

In [18]:
book = load_workbook('Q1_predictions.xlsx')
writer = pd.ExcelWriter('Q1_predictions.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

df_reasons.to_excel(writer, "rf_reason_codes", index=False)

writer.save()

# Reason Codes for Gradient Boosting Model

In [19]:
os.chdir('../Models/')

In [20]:
GBM = pickle.load(open('GBM.pickle','rb'))

In [21]:
os.chdir('../output_data/')

In [22]:
def reason_code_gbm(model,data,target='Close_Price'):
    predictors = lr_model_columns
    df_reasons = pd.DataFrame()
    df_reasons['Symbol'] = data['Symbol']
    df_reasons['Date'] = data['Date']
    df_reasons['Open Price'] = data['Open']
    df_reasons['Prev Close'] = data['Prev_Close(in 10^5)']*(10**5)
    df_reasons['Actual_Value'] = data['Close']
    df_reasons['Predicted_Value']=model.predict(np.array(data[lr_model_columns]))
    df_reasons['Expected Profit'] = df_reasons['Predicted_Value']-df_reasons['Prev Close']
    df_reasons['Actual Profit'] = df_reasons['Actual_Value'] - df_reasons['Open Price']
    df_reasons['Profit Diff'] = df_reasons['Expected Profit'] - df_reasons['Actual Profit']
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(data[model_columns])
    df_reasons['bias'] = explainer.expected_value
    contributions = pd.DataFrame(shap_values)
    contributions['mrc'] = contributions.apply(lambda x: ','.join([a+':'+str(b) for (a,b) in sorted(zip(lr_model_columns,x),key = lambda x: x[1])]),axis=1)
    df_reasons['mrc'] = contributions['mrc']
    
    return df_reasons

In [23]:
df_reason = reason_code_gbm(GBM,df_val)

In [24]:
df_reason.head()

,Symbol,Date,Open Price,Prev Close,Actual_Value,Predicted_Value,Expected Profit,Actual Profit,Profit Diff,bias,mrc
0,ADANIPORTS,2019-01-01,386.30,387.70,389.00,384.373689,-3.326311,2.70,-6.026311,1370.711168,"Prev_Close(in 10^5):-987.7189205782304,return_..."
1,ADANIPORTS,2019-01-02,388.00,389.00,381.30,387.086005,-1.913995,-6.70,4.786005,1370.711168,"Prev_Close(in 10^5):-986.6866210337246,return_..."
2,ADANIPORTS,2019-01-03,382.40,381.30,378.10,373.491676,-7.808324,-4.30,-3.508324,1370.711168,"Prev_Close(in 10^5):-993.9117812113694,return_..."
3,ADANIPORTS,2019-01-04,378.65,378.10,382.15,379.897809,1.797809,3.50,-1.702191,1370.711168,"Prev_Close(in 10^5):-996.4614698981376,return_..."
4,ADANIPORTS,2019-01-07,385.00,382.15,381.65,383.742686,1.592686,-3.35,4.942686,1370.711168,"Prev_Close(in 10^5):-993.9062375343549,return_..."


In [25]:
os.chdir('../Statistics/')

In [26]:
book = load_workbook('Q1_predictions.xlsx')
writer = pd.ExcelWriter('Q1_predictions.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

df_reason.to_excel(writer, "gbm_reason_codes", index=False)

writer.save()

# Reason Codes for linear regression model

In [27]:
os.chdir('../Models/')

In [28]:
LR = pickle.load(open('LR.pickle','rb'))

In [29]:
def reason_code_lr(model,data,target='Close'):
    df_reasons = pd.DataFrame()
    df_reasons['Symbol'] = data['Symbol']
    df_reasons['Date'] = data['Date']
    df_reasons['Open Price'] = data['Open']
    df_reasons['Prev Close'] = data['Prev_Close(in 10^5)']*(10**5)
    df_reasons['Actual_Value'] = data['Close']
    df_reasons['Predicted_Value'] = model.predict(data[lr_model_columns])
    df_reasons['Expected Profit'] = df_reasons['Predicted_Value']-df_reasons['Prev Close']
    df_reasons['Actual Profit'] = df_reasons['Actual_Value'] - df_reasons['Open Price']
    df_reasons['Profit Diff'] = df_reasons['Expected Profit'] - df_reasons['Actual Profit']
    df_reasons['bias'] = model.intercept_
    coef_array = np.array(model.coef_)
    values = np.array(data[lr_model_columns])
    contributions = np.array([coef_array*values[i,:] for i in range(values.shape[0])])
    contributions = pd.DataFrame(contributions)
    contributions['mrc'] = contributions.apply(lambda x: ','.join([a+':'+str(b) for (a,b) in sorted(zip(lr_model_columns,x),key = lambda x: x[1])]),axis=1)
    df_reasons['mrc'] = contributions['mrc']
    
    return df_reasons

In [30]:
df_reasons = reason_code_lr(LR,df_val)

In [31]:
df_reasons

,Symbol,Date,Open Price,Prev Close,Actual_Value,Predicted_Value,Expected Profit,Actual Profit,Profit Diff,bias,mrc
0,ADANIPORTS,2019-01-01,386.30,387.70,389.00,388.363440,0.663440,2.70,-2.036560,-0.159968,"return_4d:-1.5065816918734007,return_5d_fmcg_c..."
1,ADANIPORTS,2019-01-02,388.00,389.00,381.30,387.775477,-1.224523,-6.70,5.475477,-0.159968,"return_1d_realty_sqr:-0.5002741545348945,retur..."
2,ADANIPORTS,2019-01-03,382.40,381.30,378.10,379.925383,-1.374617,-4.30,2.925383,-0.159968,"return_1d:-0.8411370466197687,return_1d_media:..."
3,ADANIPORTS,2019-01-04,378.65,378.10,382.15,375.966925,-2.133075,3.50,-5.633075,-0.159968,"return_1d_media:-1.4484259459039686,return_2d_..."
4,ADANIPORTS,2019-01-07,385.00,382.15,381.65,381.752783,-0.397217,-3.35,2.952783,-0.159968,"return_2d_media:-1.60780022841768,return_3d_ni..."
5,ADANIPORTS,2019-01-08,383.00,381.65,378.75,382.513910,0.863910,-4.25,5.113910,-0.159968,"return_3d_nifty_sqr:-0.8833346152067001,return..."
6,ADANIPORTS,2019-01-09,382.00,378.75,377.30,378.649132,-0.100868,-4.70,4.599132,-0.159968,"return_1d_media:-0.8051322946792446,return_1d:..."
7,ADANIPORTS,2019-01-10,377.95,377.30,380.25,374.969061,-2.330939,2.30,-4.630939,-0.159968,"return_2d_media:-0.8937232110156951,return_5d_..."
8,ADANIPORTS,2019-01-11,380.30,380.25,377.15,379.643294,-0.606706,-3.15,2.543294,-0.159968,"return_5d_it:-0.9111167170674681,return_2d_med..."
9,ADANIPORTS,2019-01-14,377.00,377.15,374.45,377.566056,0.416056,-2.55,2.966056,-0.159968,"return_1d:-0.4034669009849625,return_1d_media:..."


In [32]:
os.chdir('../Statistics/')

In [33]:
book = load_workbook('Q1_predictions.xlsx')
writer = pd.ExcelWriter('Q1_predictions.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

df_reasons.to_excel(writer, "lr_reason_codes", index=False)

writer.save()